In [88]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#MNIST数据集相关得常数
INPUT_NODE = 784        # 输入层的节点数。对于数据集，就等于图片的像素
OUTPUT_NODE = 10        # 输出层的节点数。这个等于类别的数目，因为需要区分0~9这10个数字，所有输出层节点数为10

# 配置神经网络的参数
LAYER1_NODE = 500       # 隐藏层节点数。这里使用只有一个隐藏层的网络结果，这个隐藏层有500个节点
BATCH_SIZE = 100        # 一个训练batch中数据集的个数。

LEARNING_RATE_BASE = 0.8  # 基础的学习率
LEARNING_RATE_DECAY = 0.99  # 学习的衰减率

REGYLARIZATION_RATE = 0.0001  #描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS = 30000     # 训练轮数
MOVING_AVERAGE_DECAY = 0.99    # 滑动平均衰减率

In [99]:
# 一个辅助函数，给定神经网络的输入和所有参数，计算神经网络的前向传播结果。在这里定义一个RELU激活函数的三层全连接神经网络。
# 这个函数也支持传入用于计算参数平均值的类，这样方便在测试时使用滑动平均模型
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class == None:
        # 计算隐藏层的前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        # 计算输出层的前向传播结果。因为在计算损失函数的时候会一并计算softmax函数，
        # 所以这里不需要加入激活函数。而且不加入softmax不会影响预测结果。因为预测时，
        # 使用的是不同类别对应输出值的相对大小，有没有softmax层对最后分类结果的计算没影响。
        # 于是在计算整个神经网络的前向传播时可以不加入最后的softmax层
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 首先使用avg_class.average函数计算出变量的滑动平均值
        # 然后再计算相应的神经网络前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

In [112]:
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[ LAYER1_NODE]))
    # 输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算在当前参数下神经网络前向传播的结果。这里给出的用于计算滑动平均的类为None，
    # 所以参数不会使用参数的滑动平均
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量。这个变量不需要计算滑动平均值，所以这里指定这个变量为
    # 不可训练的变量(trainable=False)。在使用Tensorflow训练神经网络时，
    # 一般会将代表训练轮数的变量指定为不可训练的参数
    global_step = tf.Variable(0, trainable=False)
    
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类。
    # 给定训练轮数的变量能够加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    
    # 在所有神经网络参数的变量上使用滑动平均。其它辅助变量（比如global_step）就不需要了。
    # tf.trainable_variables返回的就是图上集合GraphKeys.TRAINABLE——VARIABLES中的元素。
    # 这个集合的元素就是没指定trainable=False的参数
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
     
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数。使用sparse_softmax_cross_entropy_with_logits计算
    # 函数中第一个参数是神经网络不包括softmax层的前向传播结果，第二个是训练数据的正确答案
    # 使用tf.argmax函数得到正确答案对应的类别标号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失
    regularizer = tf.contrib.layers.l2_regularizer(REGYLARIZATION_RATE)
    # 计算模型的正则化损失
    regularization = regularizer(weights1) + regularizer(weights2)
    # 总损失等于
    loss = cross_entropy_mean + regularization
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples/BATCH_SIZE, MOVING_AVERAGE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 在训练神经网络模型时，每过一边数据既需要通过反向传播来更新神经网络的参数，
    # 又要更新每一个参数的滑动平均。为了一次完成多个操作，Tensorflow提供了
    # tf.congtrol_dependencies和tf.group两种机制。下面两行程序和
    # train_op = tf.control_dependencies(train_step, variable_averages_op)
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')
    
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确.  tf.argmax(average_y, 1)
    # tf.equal判断两个张量的每一维是否相等，如果相等，则返回True
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 下面这个运算首先将一个布尔型的数值转换为实型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话 开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据集
        validate_feed = {x:mnist.validation.images, y_:mnist.validation.labels}
        # 准备测试数据
        test_feed = {x:mnist.test.images, y_:mnist.test.labels}
        # 迭代训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的预测结果
            if i%1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training steps, validation accuracy using average model is %g" % (i, validate_acc))

            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x:xs, y_:ys})

        # 在训练结束后，在测试数据集上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training steps, test accuracy using average model is %g" % (i, test_acc))

In [113]:
#载入MNIST数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
train(mnist)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
After 0 training steps, validation accuracy using average model is 0.0632
After 1000 training steps, validation accuracy using average model is 0.976
After 2000 training steps, validation accuracy using average model is 0.9816
After 3000 training steps, validation accuracy using average model is 0.9842
After 4000 training steps, validation accuracy using average model is 0.9842
After 5000 training steps, validation accuracy using average model is 0.9844
After 6000 training steps, validation accuracy using average model is 0.9852
After 7000 training steps, validation accuracy using average model is 0.9848
After 8000 training steps, validation accuracy using average model is 0.9848
After 9000 training steps, validation accuracy using average model is 0.9848
After 10000 training steps, validation a